## Análisis exploratorio de datos - Properati

**Alumnos:**
* Cabrera, Laura - 87696
* Mankowski, Daniel - 84658
* Rivas, Rafael
* Rocha, Santiago - 85726

**Link al repo en github:** https://github.com/DanielMankowski/Data-Scients

In [ ]:
%matplotlib inline

import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('default') # haciendo los graficos un poco mas bonitos xD
plt.rcParams['figure.figsize'] = (15, 5)

In [ ]:
# Levantamos los datos obtenidos de Properati
propiedades = pd.read_csv('./datasets/properati-AR-2017-08-01-properties-sell.csv', sep=',', error_bad_lines=False, low_memory=False)
propiedades.info()

In [ ]:
# Vemos un registro como ejemplo del dataframe
propiedades.sample()

### Data Cleaning
* Analizar los diferentes valores que puede tomar state_name para hacer el filtro correctamente y no perder información.

In [ ]:
propiedades.loc[:, 'state_name'].value_counts()

* Agregar columnas de Año y Mes para futuros análisis

In [ ]:
propiedades.loc[:,'year'] = pd.to_datetime(propiedades.loc[:,'created_on']).dt.year
propiedades.loc[:,'month'] = pd.to_datetime(propiedades.loc[:,'created_on']).dt.month
propiedades.sample()

* Filtrar las propiedades de CABA y GBA.

In [ ]:
# Como el enunciado lo indica nos quedamos con los datos de Capital Federal y Gran Buenos Aires
prop_caba_gba = propiedades[propiedades['state_name'].str.contains('Capital Federal') | propiedades['state_name'].str.contains('G.B.A.')]
prop_caba_gba.sample()

* Agregar una columna con el Barrio o Partido de la propiedad 

In [ ]:
def parsear_zona(row):
    mylist = row.loc['place_with_parent_names'].split("|")
    return mylist[3]

prop_caba_gba.loc[:,'neighborhood'] = prop_caba_gba.apply(parsear_zona, axis=1)   

In [ ]:
# Vista final de cómo quedó el DataFrame
prop_caba_gba.head()

## ¿Cuántas propiedades se vendieron por estado?

Entendemos como **Estado** a Capital Federal o cada una de las 3 regiones del Gran Buenos Aires. Una primera vista nos permite tener una idea de cómo se dividió la cantidad de ventas de propiedades entre estos cuatro estados.

In [ ]:
prop_barrio = prop_caba_gba['state_name'].value_counts()
prop_barrio

In [ ]:
plt.figure()
colors = ['steelblue', 'seagreen', 'palegoldenrod', 'lightcoral']
plt.title('Porcentaje de ventas por estado', fontsize=20)
prop_barrio.plot.pie(figsize=(7, 7), colors = colors, autopct='%1.2f%%', startangle=20);


Podemos observar que la mayoría de las ventas se producen en CABA en su totalidad y la Zona Norte del Gran Buenos Aires.

### ¿Cuáles son los barrio en donde se produce el mayor número de ventas?

Queremos hacer un análisis más fino sobre la distribución de ventas ya no por estados sino por barrios; entendiendo por **barrio** a un barrio de CABA propiamente dicho o una localidad del Gran Buenos Aires. Para esto vamos a usar la columna *neighborhood* que obtuvimos previamente.

In [ ]:
prop_localidad = prop_caba_gba['neighborhood'].value_counts().head(10)
prop_localidad

Si bien en el anĺisis previo la mayoría de las ventas se realiza en CABA, al hacer este segundo análisis más detallado nos encontramos que es la localidad de **Tigre** la que concentra el mayor número de transacciones.

In [ ]:
plt.title('Top ventas por barrio', fontsize=20)
plt.ylabel('Cantidad de ventas', fontsize=10)
prop_localidad.plot('bar', rot=70, color='orange')

## ¿Cuántas propiedades de cada tipo se venden?

Nos interesa analizar si hay varios tipos de propiedad a la ventas y cuántos operaciones se concretan. Esto podría ayudarnos a predecir si existe algún tipo de preferencia a la hora de buscar una propiedad o el perfil del usuario.

In [ ]:
prop_caba_gba['property_type'].value_counts()

Según los datos obtenidos, se puede observar que la gran mayoría de las publicaciones son sobre Departamentos o Casas, quedando muy por detrás los PH y los comercios.

## ¿Influye el barrio en el valor de las propiedades?

Queremos ver el valor promedio de las propiedades en relación al barrio al cual pertenecen. Para no tener valores distorsionados filtramos los barrios que tengan menos de 50 ventas.

In [ ]:
data = prop_caba_gba.loc[:,['neighborhood', 'price_aprox_usd']]
data = data[data['neighborhood'] <> '']
group_by_barrio = data.groupby('neighborhood').agg([np.mean, np.size])
group_by_barrio = group_by_barrio.loc[group_by_barrio[('price_aprox_usd','size')] > 50].sort_values(('price_aprox_usd','mean'), ascending=False)
group_by_barrio.head(10)

In [ ]:
data_barrio = prop_caba_gba.loc[:,['place_name', 'price_aprox_usd']]
data_barrio.dropna(inplace=True)
group_by_place = data_barrio.groupby('place_name').agg([np.mean, np.size])
group_by_place = group_by_place.loc[group_by_place[('price_aprox_usd','size')] > 50]\
                 .sort_values(('price_aprox_usd','mean'), ascending=False)
group_by_place.head(10)

In [ ]:
%matplotlib inline
pl = group_by_place.loc[:,('price_aprox_usd','mean')].head(20).plot('bar', title = 'Precios por barrio', figsize=(15,10))
pl.set_xlabel('Nombre del barrio', fontsize = 12)
pl.set_ylabel('Precio promedio', fontsize = 12)
pl.set_title('Precios por barrio', fontsize = 20);

Podemos observar que los valores calculados se condicen con lo que uno pensaria, ya que vemos a Palermo y Puerto Madero en los primeros puestos. Luego se ven algunos barrios cerrados en la lista seguramente porque es un negocio particular de la inmobiliaria y tienen una muestra considerable.

### Cantidad de ventas de propiedades por año y mes

In [ ]:
ventas_por_anio = prop_caba_gba.groupby('year').count()['created_on']
ventas_por_anio

In [ ]:
plt.title('Top ventas por anio', fontsize=20)
plt.ylabel('Cantidad de ventas', fontsize=12)
ventas_por_anio.plot('bar', color = 'green');

In [ ]:
ventas_por_mes = prop_caba_gba.groupby(['year', 'month']).count()['created_on']

In [ ]:
%matplotlib inline
ventas_por_mes.plot('bar', title = 'Ventas por mes por anio', figsize=(15,10));

### Tipos de propiedad vendidas

In [ ]:
ventas_por_tipo = prop_caba_gba.groupby(['year','property_type']).count()['created_on']

In [ ]:
%matplotlib inline
ventas_por_tipo.plot('bar', title = 'Ventas por tipo por anio', figsize=(15,10));

### Evolución de ventas de departamentos por año

In [ ]:
ventas_dptos = prop_caba_gba[prop_caba_gba['property_type'] == 'apartment']
ventas_dptos = ventas_dptos.groupby(['year', 'month']).count()['created_on']

In [ ]:
%matplotlib inline
ventas_dptos.plot('bar', title = 'Ventas de departamentos por anio', figsize=(15,10));

# ¿Que pasa con Tigre y Zonas Aledañas?
En graficos anteriores hemos visto que en la base de datos proporcionada por Properati tenemos una gran cantidad de ventas/publicaciones en el partido de Tigre. Debido a que uno de los integrantes del grupo vive en dicho partido, hemos investigado un poco y descubierto que la zona ha crecido mucho en los ultimos 25 años. Si bien no tenemos datos sobre los ultimos 25 años, con la base de datos propuesta haremos un analisis de la evolucion del precio del m2 en dicha zona intentando de sacar alguna tendencia. Para el analisis nos hemos propuesto comparar la evolucion de 3 zonas de Tigre ( Nordelta, Benavidez, Tigre Centro ) y de San Fernando partido aledaño a Tigre y que tambien ha crecido mucho en los ultimos años como alternativa a Tigre.
Elegimos Nordelta ya que su crecimiento ha sido realmente exponencial y creemos que influye en el precio de zonas aledañas como es Tigre y Benavidez
Eligmos San Fernando ya que entendemos que en los ultimos años ha sido buscado como alternativa a Tigre para vivir en zona norte.


In [ ]:
# Nos quedamos con las propiedades de Tigre, San Fernando, Nordelta, Benavidez
# Debido a la poca cantidad de registros en 2012 lo excluiremos del analisis
prop_zn = prop_caba_gba[(( prop_caba_gba['place_name'] == 'Nordelta' ) | ( prop_caba_gba['place_name'] == 'Tigre' ) | \
                        ( prop_caba_gba['place_name'] == 'San Fernando' ) | ( prop_caba_gba['place_name'] == 'Benavidez' )) \
                        & ( prop_caba_gba['year'] > 2013 )]

# Nos quedamos con las columnas que nos interesan y borramos si tienen datos vacios
prop_zn = prop_zn[['place_name', 'price_usd_per_m2', 'year']]
prop_zn.dropna(inplace=True)

prop_zn['place_name'].value_counts()


In [ ]:
# Hacemos un describe para ver la distribucion de los datos y si tenemos que filtrar algunos valores grandes o pequeños
# que no introduzcan ruido en nuestro analisis
print(prop_zn.describe())

In [ ]:
# De la ejecucion anterior notamos que tanto el max como el minimo son valores poco probables para 
# el precio del m2 por lo que filtraremos utilazando como criterio la distribucion de los datos
prop_zn = prop_zn[(prop_zn['price_usd_per_m2'] > 300) & prop_zn['price_usd_per_m2'] < 3500]

In [ ]:
# Veamos la comparacion de precio promedio del metro cuadrado de Tigre vs San Fernando
%matplotlib inline
plt.figure()

plt.title('Comparacion Precio Promedio', fontsize=20)
plt.ylabel('Precio m2 Promedio', fontsize=10)
plt.xlabel('Anio', fontsize=10)
prop_zn.loc[prop_zn['place_name'] == 'Tigre' , ['place_name' , 'price_usd_per_m2','year'] ]\
             .groupby('year').mean()\
             .loc[:,'price_usd_per_m2']\
             .plot.line(c='y',   rot=90, xticks=range(2014,2017), linewidth=3,figsize=(16,8), alpha=1,label='Tigre');
            
prop_zn.loc[prop_zn['place_name'] == 'San Fernando' , ['place_name' , 'price_usd_per_m2','year'] ]\
             .groupby('year').mean()\
             .loc[:,'price_usd_per_m2']\
             .plot.line(c='g',   rot=90, xticks=range(2014,2017), linewidth=3,figsize=(16,8), alpha=1,label='San Fdo.');            
            
plt.legend();            

## Datos Adicionales para utilizar en los algoritmos

Para usar en el futuro con los algoritmos de predicción hicimos una búsqueda de datos adicionales que consideramos que pueden ser de utilidad. Por ejemplo una relación entre estaciones de subte/tren, escuelas, etc y el valor de las propiedades. Esto podría ser aplicado de forma relativamente sencilla con un algortimos de KNN luego de realizar un pre procesamiento de los datos.

## Comunas

In [ ]:
comunas = pd.read_csv('datasets/comunas.csv', sep=',', error_bad_lines=False, low_memory=False)
comunas.info()

In [ ]:
comunas

## Subtes

In [ ]:
subtes = pd.read_csv('datasets/estaciones-de-subte.csv', sep=',', error_bad_lines=False, low_memory=False)
subtes.info()

In [ ]:
subtes.head(10)

## Ferrocarriles

In [ ]:
ferrocarriles_caba = pd.read_csv('datasets/estaciones-de-ferrocarril.csv', sep=';', error_bad_lines=False, low_memory=False)
ferrocarriles_caba.info()

In [ ]:
ferrocarriles_caba.head(10)

## Espacios verdes públicos

In [ ]:
espacio_verde_caba = pd.read_csv('datasets/espacio-verde-publico.csv', sep=',', error_bad_lines=False, low_memory=False)
espacio_verde_caba.info()

In [ ]:
espacio_verde_caba.head(10)

## Establecimientos educativos

In [ ]:
establecimientos_educativos = pd.read_csv('datasets/establecimientos-educativos.csv', sep=';', error_bad_lines=False, low_memory=False)
establecimientos_educativos.info()

In [ ]:
establecimientos_educativos.head(5)

## Evolución del dolar

In [ ]:
evol_dolar = pd.read_csv('datasets/evol_dolar.csv', sep=',', error_bad_lines=False, low_memory=False)
evol_dolar.info()

In [ ]:
# data_fecha = evol_dolar.set_index('Fecha')
fecha_1 = dt.datetime(2013, 1, 1).strftime("%d/%m/%y")
evol_dolar = evol_dolar[evol_dolar['Fecha'] > fecha_1]
evol_dolar["Fecha"]=pd.to_datetime(evol_dolar["Fecha"])
evol_dolar = evol_dolar[evol_dolar['Fecha'] > '2013/01/01']

In [ ]:
evol_dolar_inf = evol_dolar[evol_dolar['Tipo de d\xc3\xb3lar'] == 'Informal']
evol_dolar_inf.head(10)

In [ ]:
dolar_inf = evol_dolar_inf
dolar_inf.loc[:,'year'] = pd.to_datetime(evol_dolar_inf.loc[:,'Fecha']).dt.year
dolar_inf.loc[:,'month'] = pd.to_datetime(evol_dolar_inf.loc[:,'Fecha']).dt.month
dolar_inf.head(10)

In [ ]:
%matplotlib inline
ax = dolar_inf[['Precio de venta','Fecha']].plot(kind='line', title = 'Valor del dolar por fecha', figsize=(15,10))
ax.set_xlabel("Fecha",fontsize=12)
plt.show()